In [1]:
from keras.models import Sequential
from keras import layers
import numpy as np
from six.moves import range

Using TensorFlow backend.


# Parameters Config

In [2]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [3]:
REVERSE = True
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128

### Independent Variables 
DATA_SIZE = 30000
DIGITS = 3
MAXLEN = DIGITS + 1 + DIGITS
ansmaxlen = DIGITS + 1
chars = '0123456789+ '
LAYERS = 1

In [4]:
class CharacterTable(object):
    def __init__(self, chars):
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))
    
    def encode(self, C, num_rows):
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x
    
    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return "".join(self.indices_char[i] for i in x)

In [5]:
ctable = CharacterTable(chars)
print(ctable.chars)
print(ctable.char_indices)
print(ctable.indices_char)

[' ', '+', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
{' ': 0, '+': 1, '0': 2, '1': 3, '2': 4, '3': 5, '4': 6, '5': 7, '6': 8, '7': 9, '8': 10, '9': 11}
{0: ' ', 1: '+', 2: '0', 3: '1', 4: '2', 5: '3', 6: '4', 7: '5', 8: '6', 9: '7', 10: '8', 11: '9'}


# Data Generation

In [6]:
def DataGenerate(operator='+-', maxlen=7, digits=3, ansmaxlen=4):
    questions = []
    shows = []
    expected = []
    seen = set()
    print('Generating data...')
    while len(questions) < DATA_SIZE:
        f = lambda: int(''.join(np.random.choice(list('0123456789')) for i in range(np.random.randint(1, digits + 1))))
        fop = lambda: ''.join(np.random.choice(list(operator)) for i in range(1))
        a, b, c = f(), f(), fop()
        a, b = sorted((a, b))[::-1]
        key = tuple((a, b, c))
        if key in seen:
            continue
        seen.add(key)
        q = '{}{}{}'.format(a, c, b)
        query = q + ' ' * (maxlen - len(q))
        if c == '-':
            ans = str(a - b)
        elif c == '+':
            ans = str(a + b)
        else:
            ans = str(a * b)
        ans += ' ' * (ansmaxlen - len(ans))
        if REVERSE:
            show = query
            query = query[::-1]
        questions.append(query)
        shows.append(show)
        expected.append(ans)
    print('Total addition questions:', len(questions))
    return questions, expected, shows

In [7]:
questions, expected, shows = DataGenerate('+', maxlen=MAXLEN, digits=DIGITS, ansmaxlen=ansmaxlen)
print(shows[:5], expected[:5])

Generating data...
Total addition questions: 30000
['78+2   ', '769+78 ', '87+43  ', '408+318', '360+97 '] ['80  ', '847 ', '130 ', '726 ', '457 ']


# Processing

In [8]:
def Vectorization(questions, expected, ctable, maxlen=7, ansmaxlen=4):
    print('Vectorization...')
    x = np.zeros((len(questions), maxlen, len(chars)), dtype=np.bool)
    y = np.zeros((len(expected), ansmaxlen, len(chars)), dtype=np.bool)
    for i, sentence in enumerate(questions):
        x[i] = ctable.encode(sentence, maxlen)
    for i, sentence in enumerate(expected):
        y[i] = ctable.encode(sentence, ansmaxlen)
    return x, y
x, y = Vectorization(questions, expected, ctable, maxlen=MAXLEN, ansmaxlen=DIGITS+1)

Vectorization...


In [9]:
def shuffle(x, y, ratio=0.25):
    indices = np.arange(len(y))
    np.random.shuffle(indices)
    x = x[indices]
    y = y[indices]
    
    split = int(len(y) * ratio)
    # train_test_split
    train_x = x[split:]
    train_y = y[split:]
    test_x = x[:split]
    test_y = y[:split]

    split_at = len(train_x) - len(train_x) // 10
    (x_train, x_val) = train_x[:split_at], train_x[split_at:]
    (y_train, y_val) = train_y[:split_at], train_y[split_at:]
    return x_train, y_train, x_val, y_val, test_x, test_y

x_train, y_train, x_val, y_val, test_x, test_y = shuffle(x,y)
print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

print('Testing Data:')
print(test_x.shape)
print(test_y.shape)

print("input: ", x_train[:3], '\n\n', "label: ", y_train[:3])

Training Data:
(20250, 7, 12)
(20250, 4, 12)
Validation Data:
(2250, 7, 12)
(2250, 4, 12)
Testing Data:
(7500, 7, 12)
(7500, 4, 12)
input:  [[[False False False False False False False False False False False
    True]
  [False False False False False False  True False False False False
   False]
  [False False False False False False False False  True False False
   False]
  [False  True False False False False False False False False False
   False]
  [False False False False False False False False False  True False
   False]
  [False False False  True False False False False False False False
   False]
  [False False False False False False False False False  True False
   False]]

 [[ True False False False False False False False False False False
   False]
  [False False False False False False False False False  True False
   False]
  [False False False False False False False False False False False
    True]
  [False  True False False False False False False False False False

# Build Model

In [10]:
def BuildModel(maxlen=7, ansmaxlen=4):
    print('Build model...')

    RNN = layers.LSTM
    HIDDEN_SIZE = 128 * ansmaxlen
    BATCH_SIZE = 128
    LAYERS = 1

    model = Sequential()
    model.add(RNN(HIDDEN_SIZE, input_shape=(maxlen, len(chars))))
    model.add(layers.Reshape((ansmaxlen, 128)))
    for _ in range(LAYERS):
        model.add(RNN(HIDDEN_SIZE, return_sequences=True))
    model.add(layers.Dense(len(chars), activation='softmax'))
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    model.build()
    model.summary()
    return model

# Training part

In [11]:
model = BuildModel(maxlen=MAXLEN, ansmaxlen=ansmaxlen)
for iteration in range(100):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train, verbose=1,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 512)               1075200   
_________________________________________________________________
reshape_1 (Reshape)          (None, 4, 128)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 4, 512)            1312768   
_________________________________________________________________
dense_1 (Dense)              (None, 4, 12)             6156      
Total params: 2,394,124
Trainable params: 2,394,124
Non-trainable params: 0
_________________________________________________________________

--------------------------------------------------
Iteration 0
Train on 20250 samples, validate on 2250 samples
Epoch 1/1
20250/20250 [==============================] - 10s 510us/step - loss: 1.8648 - acc: 0.3357 - val_loss: 1.7422 - val_acc: 0.354

20250/20250 [==============================] - 4s 211us/step - loss: 0.0625 - acc: 0.9814 - val_loss: 0.0423 - val_acc: 0.9889
Q 480+11  T 491  ☑ 491 
Q 328+60  T 388  ☑ 388 
Q 613+450 T 1063 ☑ 1063
Q 262+37  T 299  ☒ 399 
Q 350+2   T 352  ☑ 352 
Q 684+1   T 685  ☑ 685 
Q 377+34  T 411  ☑ 411 
Q 262+37  T 299  ☒ 399 
Q 910+68  T 978  ☑ 978 
Q 154+3   T 157  ☑ 157 

--------------------------------------------------
Iteration 14
Train on 20250 samples, validate on 2250 samples
Epoch 1/1
20250/20250 [==============================] - 4s 209us/step - loss: 0.0224 - acc: 0.9962 - val_loss: 0.0243 - val_acc: 0.9957
Q 675+597 T 1272 ☑ 1272
Q 691+155 T 846  ☑ 846 
Q 36+0    T 36   ☑ 36  
Q 675+58  T 733  ☑ 733 
Q 823+2   T 825  ☑ 825 
Q 916+2   T 918  ☑ 918 
Q 664+95  T 759  ☑ 759 
Q 172+59  T 231  ☑ 231 
Q 262+15  T 277  ☑ 277 
Q 860+38  T 898  ☑ 898 

--------------------------------------------------
Iteration 15
Train on 20250 samples, validate on 2250 samples
Epoch 1/1
20250/20250 [=====

20250/20250 [==============================] - 4s 208us/step - loss: 0.1117 - acc: 0.9654 - val_loss: 0.0295 - val_acc: 0.9908
Q 40+27   T 67   ☑ 67  
Q 175+64  T 239  ☑ 239 
Q 224+46  T 270  ☑ 270 
Q 603+313 T 916  ☑ 916 
Q 552+76  T 628  ☑ 628 
Q 89+36   T 125  ☑ 125 
Q 62+16   T 78   ☑ 78  
Q 592+71  T 663  ☑ 663 
Q 877+135 T 1012 ☑ 1012
Q 492+1   T 493  ☑ 493 

--------------------------------------------------
Iteration 28
Train on 20250 samples, validate on 2250 samples
Epoch 1/1
20250/20250 [==============================] - 4s 209us/step - loss: 0.0099 - acc: 0.9983 - val_loss: 0.0104 - val_acc: 0.9971
Q 591+8   T 599  ☑ 599 
Q 865+407 T 1272 ☑ 1272
Q 30+23   T 53   ☑ 53  
Q 90+17   T 107  ☑ 107 
Q 65+27   T 92   ☑ 92  
Q 101+65  T 166  ☑ 166 
Q 520+81  T 601  ☑ 601 
Q 929+40  T 969  ☑ 969 
Q 945+5   T 950  ☑ 950 
Q 398+2   T 400  ☑ 400 

--------------------------------------------------
Iteration 29
Train on 20250 samples, validate on 2250 samples
Epoch 1/1
20250/20250 [=====

20250/20250 [==============================] - 4s 216us/step - loss: 0.1220 - acc: 0.9613 - val_loss: 0.0519 - val_acc: 0.9841
Q 94+8    T 102  ☑ 102 
Q 859+204 T 1063 ☑ 1063
Q 10+8    T 18   ☑ 18  
Q 470+97  T 567  ☑ 567 
Q 159+97  T 256  ☑ 256 
Q 885+23  T 908  ☑ 908 
Q 230+1   T 231  ☑ 231 
Q 951+90  T 1041 ☑ 1041
Q 69+25   T 94   ☑ 94  
Q 93+84   T 177  ☑ 177 

--------------------------------------------------
Iteration 42
Train on 20250 samples, validate on 2250 samples
Epoch 1/1
20250/20250 [==============================] - 4s 215us/step - loss: 0.0147 - acc: 0.9966 - val_loss: 0.0165 - val_acc: 0.9952
Q 860+243 T 1103 ☑ 1103
Q 435+380 T 815  ☑ 815 
Q 954+94  T 1048 ☑ 1048
Q 27+12   T 39   ☑ 39  
Q 77+61   T 138  ☑ 138 
Q 402+206 T 608  ☑ 608 
Q 957+5   T 962  ☑ 962 
Q 84+60   T 144  ☑ 144 
Q 442+87  T 529  ☑ 529 
Q 800+69  T 869  ☑ 869 

--------------------------------------------------
Iteration 43
Train on 20250 samples, validate on 2250 samples
Epoch 1/1
20250/20250 [=====

20250/20250 [==============================] - 4s 209us/step - loss: 3.2303e-04 - acc: 1.0000 - val_loss: 0.0035 - val_acc: 0.9991
Q 790+8   T 798  ☑ 798 
Q 687+340 T 1027 ☑ 1027
Q 65+7    T 72   ☑ 72  
Q 409+9   T 418  ☑ 418 
Q 53+2    T 55   ☑ 55  
Q 927+18  T 945  ☑ 945 
Q 383+42  T 425  ☑ 425 
Q 356+15  T 371  ☑ 371 
Q 45+39   T 84   ☑ 84  
Q 63+46   T 109  ☑ 109 

--------------------------------------------------
Iteration 56
Train on 20250 samples, validate on 2250 samples
Epoch 1/1
20250/20250 [==============================] - 4s 220us/step - loss: 2.8690e-04 - acc: 1.0000 - val_loss: 0.0034 - val_acc: 0.9991
Q 900+3   T 903  ☑ 903 
Q 338+7   T 345  ☑ 345 
Q 284+9   T 293  ☑ 293 
Q 486+18  T 504  ☑ 504 
Q 829+47  T 876  ☑ 876 
Q 932+555 T 1487 ☑ 1487
Q 595+72  T 667  ☑ 667 
Q 355+337 T 692  ☑ 692 
Q 687+240 T 927  ☑ 927 
Q 842+6   T 848  ☑ 848 

--------------------------------------------------
Iteration 57
Train on 20250 samples, validate on 2250 samples
Epoch 1/1
20250/2025

20250/20250 [==============================] - 4s 209us/step - loss: 8.2406e-05 - acc: 1.0000 - val_loss: 0.0028 - val_acc: 0.9991
Q 990+84  T 1074 ☑ 1074
Q 91+72   T 163  ☑ 163 
Q 454+394 T 848  ☑ 848 
Q 511+369 T 880  ☑ 880 
Q 466+34  T 500  ☑ 500 
Q 420+82  T 502  ☑ 502 
Q 314+18  T 332  ☑ 332 
Q 591+8   T 599  ☑ 599 
Q 499+174 T 673  ☑ 673 
Q 13+8    T 21   ☑ 21  

--------------------------------------------------
Iteration 70
Train on 20250 samples, validate on 2250 samples
Epoch 1/1
20250/20250 [==============================] - 4s 208us/step - loss: 7.5306e-05 - acc: 1.0000 - val_loss: 0.0028 - val_acc: 0.9991
Q 668+44  T 712  ☑ 712 
Q 204+120 T 324  ☑ 324 
Q 845+89  T 934  ☑ 934 
Q 898+5   T 903  ☑ 903 
Q 27+12   T 39   ☑ 39  
Q 58+20   T 78   ☑ 78  
Q 367+29  T 396  ☑ 396 
Q 87+47   T 134  ☑ 134 
Q 933+820 T 1753 ☑ 1753
Q 948+0   T 948  ☑ 948 

--------------------------------------------------
Iteration 71
Train on 20250 samples, validate on 2250 samples
Epoch 1/1
20250/2025

20250/20250 [==============================] - 4s 208us/step - loss: 2.4805e-05 - acc: 1.0000 - val_loss: 0.0025 - val_acc: 0.9992
Q 853+8   T 861  ☑ 861 
Q 409+9   T 418  ☑ 418 
Q 802+11  T 813  ☑ 813 
Q 966+26  T 992  ☑ 992 
Q 95+19   T 114  ☑ 114 
Q 225+107 T 332  ☑ 332 
Q 742+77  T 819  ☑ 819 
Q 528+454 T 982  ☑ 982 
Q 859+34  T 893  ☑ 893 
Q 570+507 T 1077 ☑ 1077

--------------------------------------------------
Iteration 84
Train on 20250 samples, validate on 2250 samples
Epoch 1/1
20250/20250 [==============================] - 4s 209us/step - loss: 2.3028e-05 - acc: 1.0000 - val_loss: 0.0024 - val_acc: 0.9992
Q 904+663 T 1567 ☑ 1567
Q 262+15  T 277  ☑ 277 
Q 629+329 T 958  ☑ 958 
Q 648+344 T 992  ☑ 992 
Q 752+269 T 1021 ☑ 1021
Q 777+43  T 820  ☑ 820 
Q 362+47  T 409  ☑ 409 
Q 386+37  T 423  ☑ 423 
Q 482+75  T 557  ☑ 557 
Q 922+131 T 1053 ☑ 1053

--------------------------------------------------
Iteration 85
Train on 20250 samples, validate on 2250 samples
Epoch 1/1
20250/2025

20250/20250 [==============================] - 4s 210us/step - loss: 7.6791e-06 - acc: 1.0000 - val_loss: 0.0024 - val_acc: 0.9992
Q 478+27  T 505  ☑ 505 
Q 369+1   T 370  ☑ 370 
Q 979+356 T 1335 ☑ 1335
Q 981+16  T 997  ☑ 997 
Q 320+1   T 321  ☑ 321 
Q 459+8   T 467  ☑ 467 
Q 77+52   T 129  ☑ 129 
Q 287+117 T 404  ☑ 404 
Q 753+5   T 758  ☑ 758 
Q 383+28  T 411  ☑ 411 

--------------------------------------------------
Iteration 98
Train on 20250 samples, validate on 2250 samples
Epoch 1/1
20250/20250 [==============================] - 4s 210us/step - loss: 7.0555e-06 - acc: 1.0000 - val_loss: 0.0024 - val_acc: 0.9991
Q 938+847 T 1785 ☑ 1785
Q 307+57  T 364  ☑ 364 
Q 900+3   T 903  ☑ 903 
Q 494+8   T 502  ☑ 502 
Q 576+94  T 670  ☑ 670 
Q 586+30  T 616  ☑ 616 
Q 684+7   T 691  ☑ 691 
Q 438+2   T 440  ☑ 440 
Q 607+98  T 705  ☑ 705 
Q 504+99  T 603  ☑ 603 

--------------------------------------------------
Iteration 99
Train on 20250 samples, validate on 2250 samples
Epoch 1/1
20250/2025

### Analysis
* Only addition
* 3 digits
* Training size: 20.25K
* Total training epoch: 100 epoch
* First reach 99% validation accuracy at 14 epoch

In [12]:
DIGITS = 4
MAXLEN = DIGITS + 1 + DIGITS
ansmaxlen = DIGITS + 1
chars = '0123456789+ '
DATA_SIZE = 60000

ctable = CharacterTable(chars)
questions, expected, shows = DataGenerate('+', maxlen=MAXLEN, digits=DIGITS, ansmaxlen=ansmaxlen)
x, y = Vectorization(questions, expected, ctable, maxlen=MAXLEN, ansmaxlen=ansmaxlen)
x_train, y_train, x_val, y_val, test_x, test_y = shuffle(x,y)

model = BuildModel(maxlen=MAXLEN, ansmaxlen=ansmaxlen)
for iteration in range(100):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train, verbose=1,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)

Generating data...
Total addition questions: 60000
Vectorization...
Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 640)               1671680   
_________________________________________________________________
reshape_2 (Reshape)          (None, 5, 128)            0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 5, 640)            1968640   
_________________________________________________________________
dense_2 (Dense)              (None, 5, 12)             7692      
Total params: 3,648,012
Trainable params: 3,648,012
Non-trainable params: 0
_________________________________________________________________

--------------------------------------------------
Iteration 0
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 14s 344us/s

40500/40500 [==============================] - 11s 276us/step - loss: 0.0572 - acc: 0.9813 - val_loss: 0.0360 - val_acc: 0.9886
Q 240+50    T 290   ☑ 290  
Q 8520+314  T 8834  ☑ 8834 
Q 583+3     T 586   ☑ 586  
Q 2520+6    T 2526  ☑ 2526 
Q 5552+7    T 5559  ☑ 5559 
Q 4451+258  T 4709  ☑ 4709 
Q 3622+0    T 3622  ☑ 3622 
Q 4288+457  T 4745  ☑ 4745 
Q 179+43    T 222   ☑ 222  
Q 745+62    T 807   ☑ 807  

--------------------------------------------------
Iteration 13
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 11s 276us/step - loss: 0.0246 - acc: 0.9936 - val_loss: 0.0281 - val_acc: 0.9915
Q 6400+536  T 6936  ☑ 6936 
Q 654+0     T 654   ☑ 654  
Q 773+63    T 836   ☑ 836  
Q 260+14    T 274   ☑ 274  
Q 4925+40   T 4965  ☑ 4965 
Q 3079+807  T 3886  ☑ 3886 
Q 630+559   T 1189  ☑ 1189 
Q 9617+47   T 9664  ☑ 9664 
Q 4710+6    T 4716  ☑ 4716 
Q 7482+92   T 7574  ☑ 7574 

--------------------------------------------------
Iteratio

Q 342+122   T 464   ☑ 464  
Q 4100+2399 T 6499  ☑ 6499 
Q 432+6     T 438   ☑ 438  
Q 85+71     T 156   ☑ 156  
Q 8977+9    T 8986  ☑ 8986 
Q 645+235   T 880   ☑ 880  
Q 5852+4206 T 10058 ☑ 10058
Q 719+85    T 804   ☑ 804  
Q 532+499   T 1031  ☑ 1031 
Q 118+4     T 122   ☑ 122  

--------------------------------------------------
Iteration 26
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 11s 277us/step - loss: 0.0022 - acc: 0.9997 - val_loss: 0.0131 - val_acc: 0.9960
Q 2784+6    T 2790  ☑ 2790 
Q 5633+248  T 5881  ☑ 5881 
Q 147+41    T 188   ☑ 188  
Q 8890+76   T 8966  ☑ 8966 
Q 610+429   T 1039  ☑ 1039 
Q 5567+59   T 5626  ☑ 5626 
Q 4837+9    T 4846  ☑ 4846 
Q 8426+7270 T 15696 ☒ 15697
Q 9844+487  T 10331 ☑ 10331
Q 4685+79   T 4764  ☑ 4764 

--------------------------------------------------
Iteration 27
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 11s 276us/step - 

40500/40500 [==============================] - 11s 276us/step - loss: 4.5169e-04 - acc: 1.0000 - val_loss: 0.0041 - val_acc: 0.9987
Q 112+8     T 120   ☑ 120  
Q 241+170   T 411   ☑ 411  
Q 9320+49   T 9369  ☑ 9369 
Q 591+18    T 609   ☑ 609  
Q 806+160   T 966   ☑ 966  
Q 5872+24   T 5896  ☑ 5896 
Q 7692+377  T 8069  ☑ 8069 
Q 707+1     T 708   ☑ 708  
Q 6814+9    T 6823  ☑ 6823 
Q 29+20     T 49    ☑ 49   

--------------------------------------------------
Iteration 40
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 11s 277us/step - loss: 3.2829e-04 - acc: 1.0000 - val_loss: 0.0037 - val_acc: 0.9989
Q 26+15     T 41    ☑ 41   
Q 352+341   T 693   ☑ 693  
Q 3819+8    T 3827  ☑ 3827 
Q 2517+2062 T 4579  ☑ 4579 
Q 881+736   T 1617  ☑ 1617 
Q 6165+391  T 6556  ☑ 6556 
Q 42+4      T 46    ☑ 46   
Q 3934+694  T 4628  ☑ 4628 
Q 620+57    T 677   ☑ 677  
Q 277+25    T 302   ☑ 302  

--------------------------------------------------


Q 3544+0    T 3544  ☑ 3544 
Q 354+82    T 436   ☑ 436  
Q 2187+982  T 3169  ☒ 3179 
Q 8341+8    T 8349  ☑ 8349 
Q 846+4     T 850   ☑ 850  
Q 6156+79   T 6235  ☑ 6235 
Q 466+278   T 744   ☑ 744  
Q 778+215   T 993   ☑ 993  
Q 584+20    T 604   ☑ 604  
Q 8176+202  T 8378  ☑ 8378 

--------------------------------------------------
Iteration 53
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 11s 277us/step - loss: 0.0089 - acc: 0.9970 - val_loss: 0.0249 - val_acc: 0.9911
Q 79+33     T 112   ☑ 112  
Q 6498+74   T 6572  ☑ 6572 
Q 728+34    T 762   ☑ 762  
Q 3858+4    T 3862  ☑ 3862 
Q 46+15     T 61    ☑ 61   
Q 972+14    T 986   ☑ 986  
Q 724+92    T 816   ☑ 816  
Q 3625+233  T 3858  ☑ 3858 
Q 9639+965  T 10604 ☑ 10604
Q 2352+1100 T 3452  ☑ 3452 

--------------------------------------------------
Iteration 54
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 11s 280us/step - 

40500/40500 [==============================] - 11s 277us/step - loss: 0.0016 - acc: 0.9996 - val_loss: 0.0062 - val_acc: 0.9983
Q 7860+4    T 7864  ☑ 7864 
Q 5484+19   T 5503  ☑ 5503 
Q 2627+2    T 2629  ☑ 2629 
Q 933+656   T 1589  ☑ 1589 
Q 304+3     T 307   ☑ 307  
Q 194+65    T 259   ☑ 259  
Q 9134+480  T 9614  ☑ 9614 
Q 4189+520  T 4709  ☑ 4709 
Q 3243+85   T 3328  ☑ 3328 
Q 8876+175  T 9051  ☑ 9051 

--------------------------------------------------
Iteration 67
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 11s 278us/step - loss: 0.0091 - acc: 0.9973 - val_loss: 0.0395 - val_acc: 0.9869
Q 4420+33   T 4453  ☑ 4453 
Q 1803+637  T 2440  ☑ 2440 
Q 552+499   T 1051  ☑ 1051 
Q 6939+4612 T 11551 ☒ 11541
Q 6473+0    T 6473  ☑ 6473 
Q 8623+592  T 9215  ☑ 9215 
Q 78+1      T 79    ☑ 79   
Q 4817+7    T 4824  ☑ 4824 
Q 5906+2650 T 8556  ☑ 8556 
Q 962+7     T 969   ☑ 969  

--------------------------------------------------
Iteratio

40500/40500 [==============================] - 11s 276us/step - loss: 7.3195e-05 - acc: 1.0000 - val_loss: 0.0031 - val_acc: 0.9989
Q 319+1     T 320   ☑ 320  
Q 1573+1036 T 2609  ☑ 2609 
Q 4008+4    T 4012  ☑ 4012 
Q 91+40     T 131   ☑ 131  
Q 2954+195  T 3149  ☑ 3149 
Q 893+5     T 898   ☑ 898  
Q 249+8     T 257   ☑ 257  
Q 519+357   T 876   ☑ 876  
Q 783+40    T 823   ☑ 823  
Q 274+6     T 280   ☑ 280  

--------------------------------------------------
Iteration 80
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 11s 277us/step - loss: 6.2815e-05 - acc: 1.0000 - val_loss: 0.0030 - val_acc: 0.9989
Q 9876+5    T 9881  ☑ 9881 
Q 1549+725  T 2274  ☑ 2274 
Q 2577+21   T 2598  ☑ 2598 
Q 683+24    T 707   ☑ 707  
Q 459+44    T 503   ☑ 503  
Q 7917+13   T 7930  ☑ 7930 
Q 589+8     T 597   ☑ 597  
Q 2084+39   T 2123  ☑ 2123 
Q 1465+659  T 2124  ☑ 2124 
Q 6205+404  T 6609  ☑ 6609 

--------------------------------------------------


40500/40500 [==============================] - 11s 277us/step - loss: 1.0645e-05 - acc: 1.0000 - val_loss: 0.0029 - val_acc: 0.9992
Q 692+5     T 697   ☑ 697  
Q 701+68    T 769   ☑ 769  
Q 6891+710  T 7601  ☑ 7601 
Q 22+6      T 28    ☑ 28   
Q 6360+64   T 6424  ☑ 6424 
Q 1579+2    T 1581  ☑ 1581 
Q 94+24     T 118   ☑ 118  
Q 908+85    T 993   ☑ 993  
Q 4797+0    T 4797  ☑ 4797 
Q 930+535   T 1465  ☑ 1465 

--------------------------------------------------
Iteration 93
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 11s 276us/step - loss: 9.1559e-06 - acc: 1.0000 - val_loss: 0.0029 - val_acc: 0.9992
Q 8473+0    T 8473  ☑ 8473 
Q 375+9     T 384   ☑ 384  
Q 2938+7    T 2945  ☑ 2945 
Q 9028+37   T 9065  ☑ 9065 
Q 1787+327  T 2114  ☑ 2114 
Q 310+7     T 317   ☑ 317  
Q 752+475   T 1227  ☑ 1227 
Q 9970+33   T 10003 ☑ 10003
Q 6404+0    T 6404  ☑ 6404 
Q 474+122   T 596   ☑ 596  

--------------------------------------------------


### Analysis
* Only addition
* 4 digits
* Training size: 40.5K
* Total training epoch: 100 epoch
* First reach 99% validation accuracy at 13 epoch

In [13]:
DIGITS = 3
MAXLEN = DIGITS + 1 + DIGITS
ansmaxlen = DIGITS
chars = '0123456789- '
DATA_SIZE = 30000

ctable = CharacterTable(chars)
questions, expected, shows = DataGenerate('-', maxlen=MAXLEN, digits=DIGITS, ansmaxlen=ansmaxlen)
x, y = Vectorization(questions, expected, ctable, maxlen=MAXLEN, ansmaxlen=ansmaxlen)
x_train, y_train, x_val, y_val, test_x, test_y = shuffle(x,y)

model = BuildModel(maxlen=MAXLEN, ansmaxlen=ansmaxlen)
for iteration in range(100):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train, verbose=1,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)

Generating data...
Total addition questions: 30000
Vectorization...
Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 384)               609792    
_________________________________________________________________
reshape_3 (Reshape)          (None, 3, 128)            0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 3, 384)            787968    
_________________________________________________________________
dense_3 (Dense)              (None, 3, 12)             4620      
Total params: 1,402,380
Trainable params: 1,402,380
Non-trainable params: 0
_________________________________________________________________

--------------------------------------------------
Iteration 0
Train on 20250 samples, validate on 2250 samples
Epoch 1/1
20250/20250 [==============================] - 6s 289us/st

20250/20250 [==============================] - 4s 182us/step - loss: 0.1484 - acc: 0.9555 - val_loss: 0.1151 - val_acc: 0.9656
Q 812-54  T 758 ☑ 758
Q 984-83  T 901 ☑ 901
Q 810-26  T 784 ☑ 784
Q 190-81  T 109 ☑ 109
Q 497-33  T 464 ☑ 464
Q 271-8   T 263 ☑ 263
Q 879-89  T 790 ☑ 790
Q 739-73  T 666 ☑ 666
Q 371-90  T 281 ☑ 281
Q 473-72  T 401 ☑ 401

--------------------------------------------------
Iteration 14
Train on 20250 samples, validate on 2250 samples
Epoch 1/1
20250/20250 [==============================] - 4s 184us/step - loss: 0.1076 - acc: 0.9703 - val_loss: 0.1253 - val_acc: 0.9627
Q 364-26  T 338 ☑ 338
Q 959-44  T 915 ☑ 915
Q 315-3   T 312 ☑ 312
Q 520-37  T 483 ☑ 483
Q 186-26  T 160 ☑ 160
Q 99-30   T 69  ☑ 69 
Q 131-83  T 48  ☒ 49 
Q 620-94  T 526 ☑ 526
Q 512-12  T 500 ☒ 400
Q 105-3   T 102 ☑ 102

--------------------------------------------------
Iteration 15
Train on 20250 samples, validate on 2250 samples
Epoch 1/1
20250/20250 [==============================] - 4s 182us/st

20250/20250 [==============================] - 4s 182us/step - loss: 0.0945 - acc: 0.9710 - val_loss: 0.1950 - val_acc: 0.9326
Q 270-50  T 220 ☒ 210
Q 446-97  T 349 ☑ 349
Q 672-12  T 660 ☑ 660
Q 80-30   T 50  ☑ 50 
Q 947-3   T 944 ☑ 944
Q 247-5   T 242 ☑ 242
Q 148-13  T 135 ☑ 135
Q 2-1     T 1   ☒ 0  
Q 957-323 T 634 ☑ 634
Q 895-0   T 895 ☑ 895

--------------------------------------------------
Iteration 29
Train on 20250 samples, validate on 2250 samples
Epoch 1/1
20250/20250 [==============================] - 4s 182us/step - loss: 0.0663 - acc: 0.9803 - val_loss: 0.0523 - val_acc: 0.9844
Q 610-4   T 606 ☑ 606
Q 632-37  T 595 ☑ 595
Q 394-77  T 317 ☑ 317
Q 972-68  T 904 ☑ 904
Q 181-4   T 177 ☑ 177
Q 590-66  T 524 ☑ 524
Q 742-514 T 228 ☑ 228
Q 714-9   T 705 ☑ 705
Q 117-6   T 111 ☑ 111
Q 926-7   T 919 ☑ 919

--------------------------------------------------
Iteration 30
Train on 20250 samples, validate on 2250 samples
Epoch 1/1
20250/20250 [==============================] - 4s 183us/st

20250/20250 [==============================] - 4s 183us/step - loss: 0.0128 - acc: 0.9984 - val_loss: 0.0178 - val_acc: 0.9964
Q 343-32  T 311 ☑ 311
Q 258-170 T 88  ☑ 88 
Q 938-71  T 867 ☑ 867
Q 71-29   T 42  ☑ 42 
Q 689-47  T 642 ☑ 642
Q 939-20  T 919 ☑ 919
Q 662-47  T 615 ☑ 615
Q 390-0   T 390 ☑ 390
Q 787-209 T 578 ☑ 578
Q 36-0    T 36  ☑ 36 

--------------------------------------------------
Iteration 44
Train on 20250 samples, validate on 2250 samples
Epoch 1/1
20250/20250 [==============================] - 4s 182us/step - loss: 0.0105 - acc: 0.9990 - val_loss: 0.0171 - val_acc: 0.9966
Q 261-3   T 258 ☑ 258
Q 910-251 T 659 ☑ 659
Q 105-2   T 103 ☑ 103
Q 48-39   T 9   ☑ 9  
Q 257-6   T 251 ☑ 251
Q 682-350 T 332 ☑ 332
Q 682-92  T 590 ☑ 590
Q 791-8   T 783 ☑ 783
Q 605-417 T 188 ☑ 188
Q 799-3   T 796 ☑ 796

--------------------------------------------------
Iteration 45
Train on 20250 samples, validate on 2250 samples
Epoch 1/1
20250/20250 [==============================] - 4s 181us/st

20250/20250 [==============================] - 4s 181us/step - loss: 0.0039 - acc: 0.9998 - val_loss: 0.0144 - val_acc: 0.9954
Q 813-60  T 753 ☑ 753
Q 954-16  T 938 ☑ 938
Q 578-322 T 256 ☑ 256
Q 92-75   T 17  ☑ 17 
Q 809-5   T 804 ☑ 804
Q 84-81   T 3   ☑ 3  
Q 388-373 T 15  ☑ 15 
Q 386-81  T 305 ☑ 305
Q 292-30  T 262 ☑ 262
Q 959-86  T 873 ☑ 873

--------------------------------------------------
Iteration 59
Train on 20250 samples, validate on 2250 samples
Epoch 1/1
20250/20250 [==============================] - 4s 182us/step - loss: 0.1210 - acc: 0.9636 - val_loss: 0.0333 - val_acc: 0.9892
Q 25-1    T 24  ☑ 24 
Q 407-0   T 407 ☑ 407
Q 615-573 T 42  ☑ 42 
Q 455-9   T 446 ☑ 446
Q 458-71  T 387 ☑ 387
Q 93-85   T 8   ☑ 8  
Q 546-3   T 543 ☑ 543
Q 768-243 T 525 ☑ 525
Q 520-7   T 513 ☑ 513
Q 690-17  T 673 ☑ 673

--------------------------------------------------
Iteration 60
Train on 20250 samples, validate on 2250 samples
Epoch 1/1
20250/20250 [==============================] - 4s 183us/st

20250/20250 [==============================] - 4s 182us/step - loss: 0.0022 - acc: 1.0000 - val_loss: 0.0091 - val_acc: 0.9982
Q 601-26  T 575 ☑ 575
Q 276-8   T 268 ☑ 268
Q 819-194 T 625 ☑ 625
Q 727-273 T 454 ☑ 454
Q 362-1   T 361 ☑ 361
Q 620-0   T 620 ☑ 620
Q 560-29  T 531 ☑ 531
Q 591-40  T 551 ☑ 551
Q 739-73  T 666 ☑ 666
Q 592-86  T 506 ☑ 506

--------------------------------------------------
Iteration 74
Train on 20250 samples, validate on 2250 samples
Epoch 1/1
20250/20250 [==============================] - 4s 182us/step - loss: 0.0020 - acc: 1.0000 - val_loss: 0.0093 - val_acc: 0.9976
Q 470-4   T 466 ☑ 466
Q 425-36  T 389 ☑ 389
Q 83-38   T 45  ☑ 45 
Q 71-51   T 20  ☑ 20 
Q 829-0   T 829 ☑ 829
Q 767-54  T 713 ☑ 713
Q 166-29  T 137 ☑ 137
Q 832-73  T 759 ☑ 759
Q 996-77  T 919 ☑ 919
Q 957-26  T 931 ☑ 931

--------------------------------------------------
Iteration 75
Train on 20250 samples, validate on 2250 samples
Epoch 1/1
20250/20250 [==============================] - 4s 184us/st

20250/20250 [==============================] - 4s 183us/step - loss: 0.0015 - acc: 1.0000 - val_loss: 0.0082 - val_acc: 0.9982
Q 461-78  T 383 ☑ 383
Q 134-58  T 76  ☑ 76 
Q 684-6   T 678 ☑ 678
Q 946-3   T 943 ☑ 943
Q 107-58  T 49  ☑ 49 
Q 303-2   T 301 ☑ 301
Q 904-0   T 904 ☑ 904
Q 234-73  T 161 ☑ 161
Q 749-185 T 564 ☑ 564
Q 210-63  T 147 ☑ 147

--------------------------------------------------
Iteration 89
Train on 20250 samples, validate on 2250 samples
Epoch 1/1
20250/20250 [==============================] - 4s 183us/step - loss: 0.0013 - acc: 1.0000 - val_loss: 0.0081 - val_acc: 0.9981
Q 129-66  T 63  ☑ 63 
Q 995-36  T 959 ☑ 959
Q 627-24  T 603 ☑ 603
Q 685-3   T 682 ☑ 682
Q 71-67   T 4   ☑ 4  
Q 690-17  T 673 ☑ 673
Q 815-6   T 809 ☑ 809
Q 242-52  T 190 ☑ 190
Q 96-88   T 8   ☑ 8  
Q 95-37   T 58  ☑ 58 

--------------------------------------------------
Iteration 90
Train on 20250 samples, validate on 2250 samples
Epoch 1/1
20250/20250 [==============================] - 4s 183us/st

### Analysis
* Only subtraction
* 3 digits
* Training size: 20.25K
* Total training epoch: 100 epoch
* First reach 99% validation accuracy at 27 epoch

In [14]:
DIGITS = 4
MAXLEN = DIGITS + 1 + DIGITS
chars = '0123456789- '
ansmaxlen = DIGITS
DATA_SIZE = 60000

ctable = CharacterTable(chars)
questions, expected, shows = DataGenerate('-', maxlen=MAXLEN, digits=DIGITS, ansmaxlen=ansmaxlen)
x, y = Vectorization(questions, expected, ctable, maxlen=MAXLEN, ansmaxlen=ansmaxlen)
x_train, y_train, x_val, y_val, test_x, test_y = shuffle(x,y)

model = BuildModel(maxlen=MAXLEN, ansmaxlen=ansmaxlen)
for iteration in range(100):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train, verbose=1,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)

Generating data...
Total addition questions: 60000
Vectorization...
Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 512)               1075200   
_________________________________________________________________
reshape_4 (Reshape)          (None, 4, 128)            0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 4, 512)            1312768   
_________________________________________________________________
dense_4 (Dense)              (None, 4, 12)             6156      
Total params: 2,394,124
Trainable params: 2,394,124
Non-trainable params: 0
_________________________________________________________________

--------------------------------------------------
Iteration 0
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 12s 302us/s

40500/40500 [==============================] - 10s 249us/step - loss: 0.0845 - acc: 0.9721 - val_loss: 0.1099 - val_acc: 0.9593
Q 7958-81   T 7877 ☑ 7877
Q 4191-72   T 4119 ☑ 4119
Q 7592-59   T 7533 ☑ 7533
Q 3602-2    T 3600 ☑ 3600
Q 916-11    T 905  ☑ 905 
Q 437-292   T 145  ☑ 145 
Q 2443-41   T 2402 ☑ 2402
Q 466-40    T 426  ☑ 426 
Q 7774-5419 T 2355 ☒ 2256
Q 939-8     T 931  ☑ 931 

--------------------------------------------------
Iteration 13
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 9s 232us/step - loss: 0.0620 - acc: 0.9806 - val_loss: 0.0717 - val_acc: 0.9767
Q 802-80    T 722  ☑ 722 
Q 876-8     T 868  ☑ 868 
Q 133-114   T 19   ☑ 19  
Q 5527-111  T 5416 ☒ 5516
Q 7906-8    T 7898 ☑ 7898
Q 3243-39   T 3204 ☑ 3204
Q 753-700   T 53   ☒ 43  
Q 90-46     T 44   ☑ 44  
Q 3034-4    T 3030 ☑ 3030
Q 7251-0    T 7251 ☑ 7251

--------------------------------------------------
Iteration 14
Train on 40500 samples, validate on 

40500/40500 [==============================] - 9s 232us/step - loss: 0.0416 - acc: 0.9872 - val_loss: 0.0623 - val_acc: 0.9793
Q 931-95    T 836  ☑ 836 
Q 1658-101  T 1557 ☑ 1557
Q 2513-221  T 2292 ☑ 2292
Q 9869-22   T 9847 ☑ 9847
Q 4720-343  T 4377 ☑ 4377
Q 320-6     T 314  ☑ 314 
Q 8998-299  T 8699 ☑ 8699
Q 7945-2770 T 5175 ☑ 5175
Q 734-7     T 727  ☑ 727 
Q 421-23    T 398  ☑ 398 

--------------------------------------------------
Iteration 27
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 9s 232us/step - loss: 0.0377 - acc: 0.9884 - val_loss: 0.0361 - val_acc: 0.9881
Q 278-73    T 205  ☑ 205 
Q 8057-8    T 8049 ☑ 8049
Q 2049-0    T 2049 ☑ 2049
Q 755-184   T 571  ☑ 571 
Q 6348-6    T 6342 ☑ 6342
Q 624-206   T 418  ☑ 418 
Q 8131-0    T 8131 ☑ 8131
Q 8787-6705 T 2082 ☑ 2082
Q 8740-60   T 8680 ☑ 8680
Q 857-138   T 719  ☑ 719 

--------------------------------------------------
Iteration 28
Train on 40500 samples, validate on 4

40500/40500 [==============================] - 9s 232us/step - loss: 0.0338 - acc: 0.9898 - val_loss: 0.0776 - val_acc: 0.9742
Q 7904-56   T 7848 ☑ 7848
Q 9090-98   T 8992 ☒ 8002
Q 6334-241  T 6093 ☑ 6093
Q 84-60     T 24   ☑ 24  
Q 72-22     T 50   ☑ 50  
Q 7209-1016 T 6193 ☑ 6193
Q 2283-11   T 2272 ☑ 2272
Q 605-37    T 568  ☑ 568 
Q 6462-61   T 6401 ☑ 6401
Q 803-447   T 356  ☑ 356 

--------------------------------------------------
Iteration 41
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 9s 232us/step - loss: 0.0131 - acc: 0.9965 - val_loss: 0.0227 - val_acc: 0.9939
Q 7396-96   T 7300 ☑ 7300
Q 916-6     T 910  ☑ 910 
Q 460-64    T 396  ☑ 396 
Q 9866-9    T 9857 ☑ 9857
Q 774-199   T 575  ☑ 575 
Q 4304-21   T 4283 ☑ 4283
Q 959-8     T 951  ☑ 951 
Q 5017-2    T 5015 ☑ 5015
Q 7779-13   T 7766 ☑ 7766
Q 84-50     T 34   ☑ 34  

--------------------------------------------------
Iteration 42
Train on 40500 samples, validate on 4

40500/40500 [==============================] - 9s 231us/step - loss: 9.3455e-04 - acc: 0.9999 - val_loss: 0.0159 - val_acc: 0.9958
Q 223-135   T 88   ☑ 88  
Q 103-5     T 98   ☑ 98  
Q 45-45     T 0    ☑ 0   
Q 97-63     T 34   ☑ 34  
Q 7777-2323 T 5454 ☑ 5454
Q 8975-403  T 8572 ☑ 8572
Q 3686-451  T 3235 ☑ 3235
Q 541-428   T 113  ☑ 113 
Q 476-12    T 464  ☑ 464 
Q 42-23     T 19   ☑ 19  

--------------------------------------------------
Iteration 55
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 9s 232us/step - loss: 0.0595 - acc: 0.9820 - val_loss: 0.0628 - val_acc: 0.9798
Q 7448-135  T 7313 ☑ 7313
Q 176-2     T 174  ☑ 174 
Q 934-9     T 925  ☑ 925 
Q 5469-91   T 5378 ☑ 5378
Q 5500-8    T 5492 ☑ 5492
Q 567-1     T 566  ☑ 566 
Q 2296-2    T 2294 ☑ 2294
Q 7930-81   T 7849 ☑ 7849
Q 591-20    T 571  ☑ 571 
Q 2156-6    T 2150 ☑ 2150

--------------------------------------------------
Iteration 56
Train on 40500 samples, validate 

40500/40500 [==============================] - 9s 233us/step - loss: 0.0040 - acc: 0.9990 - val_loss: 0.0236 - val_acc: 0.9932
Q 584-425   T 159  ☑ 159 
Q 672-74    T 598  ☑ 598 
Q 257-34    T 223  ☑ 223 
Q 882-38    T 844  ☑ 844 
Q 1996-9    T 1987 ☑ 1987
Q 71-37     T 34   ☑ 34  
Q 6082-0    T 6082 ☑ 6082
Q 574-47    T 527  ☑ 527 
Q 961-437   T 524  ☑ 524 
Q 7606-541  T 7065 ☑ 7065

--------------------------------------------------
Iteration 69
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 9s 232us/step - loss: 0.0128 - acc: 0.9962 - val_loss: 0.0532 - val_acc: 0.9838
Q 768-668   T 100  ☑ 100 
Q 7240-389  T 6851 ☑ 6851
Q 564-34    T 530  ☑ 530 
Q 6407-776  T 5631 ☑ 5631
Q 248-11    T 237  ☑ 237 
Q 9433-7555 T 1878 ☑ 1878
Q 8706-955  T 7751 ☑ 7751
Q 61-3      T 58   ☑ 58  
Q 7441-714  T 6727 ☑ 6727
Q 4344-5    T 4339 ☑ 4339

--------------------------------------------------
Iteration 70
Train on 40500 samples, validate on 4

40500/40500 [==============================] - 9s 232us/step - loss: 0.0100 - acc: 0.9971 - val_loss: 0.0224 - val_acc: 0.9933
Q 8658-818  T 7840 ☑ 7840
Q 83-47     T 36   ☑ 36  
Q 405-28    T 377  ☑ 377 
Q 7721-0    T 7721 ☑ 7721
Q 6711-260  T 6451 ☑ 6451
Q 879-665   T 214  ☑ 214 
Q 814-80    T 734  ☑ 734 
Q 260-3     T 257  ☑ 257 
Q 1291-1    T 1290 ☑ 1290
Q 580-53    T 527  ☑ 527 

--------------------------------------------------
Iteration 83
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 9s 232us/step - loss: 0.0040 - acc: 0.9990 - val_loss: 0.0222 - val_acc: 0.9939
Q 1091-682  T 409  ☑ 409 
Q 6080-1347 T 4733 ☑ 4733
Q 4146-0    T 4146 ☑ 4146
Q 6544-4314 T 2230 ☑ 2230
Q 901-1     T 900  ☑ 900 
Q 741-59    T 682  ☑ 682 
Q 1382-47   T 1335 ☑ 1335
Q 84-52     T 32   ☑ 32  
Q 9197-7605 T 1592 ☒ 1492
Q 287-56    T 231  ☑ 231 

--------------------------------------------------
Iteration 84
Train on 40500 samples, validate on 4

40500/40500 [==============================] - 9s 231us/step - loss: 0.0084 - acc: 0.9977 - val_loss: 0.0257 - val_acc: 0.9921
Q 1587-1    T 1586 ☑ 1586
Q 1155-633  T 522  ☑ 522 
Q 9003-89   T 8914 ☑ 8914
Q 9942-2    T 9940 ☑ 9940
Q 7366-6    T 7360 ☑ 7360
Q 3659-0    T 3659 ☑ 3659
Q 3500-18   T 3482 ☑ 3482
Q 2777-971  T 1806 ☑ 1806
Q 3491-90   T 3401 ☑ 3401
Q 532-8     T 524  ☑ 524 

--------------------------------------------------
Iteration 97
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 9s 232us/step - loss: 0.0011 - acc: 0.9998 - val_loss: 0.0163 - val_acc: 0.9955
Q 1956-2    T 1954 ☑ 1954
Q 731-1     T 730  ☑ 730 
Q 146-63    T 83   ☑ 83  
Q 8508-8    T 8500 ☑ 8500
Q 582-198   T 384  ☑ 384 
Q 696-71    T 625  ☑ 625 
Q 213-68    T 145  ☑ 145 
Q 24-10     T 14   ☑ 14  
Q 7396-96   T 7300 ☑ 7300
Q 5440-456  T 4984 ☑ 4984

--------------------------------------------------
Iteration 98
Train on 40500 samples, validate on 4

### Analysis
* Only subtraction
* 4 digits
* Training size: 40.5K
* Total training epoch: 100 epoch
* First reach 99% validation accuracy at 25 epoch

In [15]:
DIGITS = 3
MAXLEN = DIGITS + 1 + DIGITS
chars = '0123456789+- '
ansmaxlen = DIGITS + 1
DATA_SIZE = 45000

ctable = CharacterTable(chars)
questions, expected, shows = DataGenerate('+-', maxlen=MAXLEN, digits=DIGITS, ansmaxlen=ansmaxlen)
x, y = Vectorization(questions, expected, ctable, maxlen=MAXLEN, ansmaxlen=ansmaxlen)
x_train, y_train, x_val, y_val, test_x, test_y = shuffle(x,y)

model = BuildModel(maxlen=MAXLEN, ansmaxlen=ansmaxlen)
for iteration in range(100):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train, verbose=1,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)

Generating data...
Total addition questions: 45000
Vectorization...
Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_9 (LSTM)                (None, 512)               1077248   
_________________________________________________________________
reshape_5 (Reshape)          (None, 4, 128)            0         
_________________________________________________________________
lstm_10 (LSTM)               (None, 4, 512)            1312768   
_________________________________________________________________
dense_5 (Dense)              (None, 4, 13)             6669      
Total params: 2,396,685
Trainable params: 2,396,685
Non-trainable params: 0
_________________________________________________________________

--------------------------------------------------
Iteration 0
Train on 30375 samples, validate on 3375 samples
Epoch 1/1
30375/30375 [==============================] - 10s 324us/s

30375/30375 [==============================] - 6s 210us/step - loss: 0.0638 - acc: 0.9821 - val_loss: 0.1032 - val_acc: 0.9649
Q 639-84  T 555  ☑ 555 
Q 528+71  T 599  ☒ 609 
Q 112-66  T 46   ☒ 55  
Q 63-1    T 62   ☑ 62  
Q 816+36  T 852  ☑ 852 
Q 695+184 T 879  ☑ 879 
Q 721+4   T 725  ☑ 725 
Q 470-8   T 462  ☑ 462 
Q 148+38  T 186  ☑ 186 
Q 508+20  T 528  ☑ 528 

--------------------------------------------------
Iteration 14
Train on 30375 samples, validate on 3375 samples
Epoch 1/1
30375/30375 [==============================] - 6s 210us/step - loss: 0.0529 - acc: 0.9851 - val_loss: 0.0666 - val_acc: 0.9787
Q 346-129 T 217  ☑ 217 
Q 79+26   T 105  ☑ 105 
Q 893+784 T 1677 ☑ 1677
Q 73-14   T 59   ☒ 69  
Q 835-73  T 762  ☑ 762 
Q 611-4   T 607  ☑ 607 
Q 764-0   T 764  ☑ 764 
Q 715-571 T 144  ☑ 144 
Q 949-176 T 773  ☑ 773 
Q 689+205 T 894  ☑ 894 

--------------------------------------------------
Iteration 15
Train on 30375 samples, validate on 3375 samples
Epoch 1/1
30375/30375 [=====

30375/30375 [==============================] - 6s 210us/step - loss: 0.0270 - acc: 0.9923 - val_loss: 0.0691 - val_acc: 0.9776
Q 63-13   T 50   ☑ 50  
Q 721+91  T 812  ☑ 812 
Q 920-605 T 315  ☑ 315 
Q 471-72  T 399  ☑ 399 
Q 487+48  T 535  ☑ 535 
Q 203-7   T 196  ☑ 196 
Q 904+815 T 1719 ☑ 1719
Q 458-18  T 440  ☑ 440 
Q 63-13   T 50   ☑ 50  
Q 207+4   T 211  ☑ 211 

--------------------------------------------------
Iteration 28
Train on 30375 samples, validate on 3375 samples
Epoch 1/1
30375/30375 [==============================] - 6s 209us/step - loss: 0.0748 - acc: 0.9762 - val_loss: 0.0876 - val_acc: 0.9700
Q 40-33   T 7    ☑ 7   
Q 529-424 T 105  ☑ 105 
Q 92+36   T 128  ☑ 128 
Q 701+698 T 1399 ☒ 1499
Q 895+623 T 1518 ☑ 1518
Q 78-2    T 76   ☑ 76  
Q 639-84  T 555  ☑ 555 
Q 560-85  T 475  ☑ 475 
Q 183-24  T 159  ☑ 159 
Q 586-26  T 560  ☑ 560 

--------------------------------------------------
Iteration 29
Train on 30375 samples, validate on 3375 samples
Epoch 1/1
30375/30375 [=====

30375/30375 [==============================] - 6s 210us/step - loss: 0.1088 - acc: 0.9673 - val_loss: 0.0405 - val_acc: 0.9878
Q 946+13  T 959  ☑ 959 
Q 95-50   T 45   ☑ 45  
Q 281-7   T 274  ☑ 274 
Q 831-1   T 830  ☑ 830 
Q 997-303 T 694  ☒ 594 
Q 874-15  T 859  ☑ 859 
Q 94-40   T 54   ☑ 54  
Q 816+3   T 819  ☑ 819 
Q 728+71  T 799  ☑ 799 
Q 763-3   T 760  ☑ 760 

--------------------------------------------------
Iteration 42
Train on 30375 samples, validate on 3375 samples
Epoch 1/1
30375/30375 [==============================] - 6s 210us/step - loss: 0.0089 - acc: 0.9986 - val_loss: 0.0218 - val_acc: 0.9934
Q 421-0   T 421  ☑ 421 
Q 313-55  T 258  ☑ 258 
Q 570+3   T 573  ☑ 573 
Q 347-6   T 341  ☑ 341 
Q 62-23   T 39   ☑ 39  
Q 28+2    T 30   ☑ 30  
Q 257-4   T 253  ☑ 253 
Q 305+9   T 314  ☑ 314 
Q 783+47  T 830  ☑ 830 
Q 690+22  T 712  ☑ 712 

--------------------------------------------------
Iteration 43
Train on 30375 samples, validate on 3375 samples
Epoch 1/1
30375/30375 [=====

30375/30375 [==============================] - 6s 210us/step - loss: 0.0012 - acc: 1.0000 - val_loss: 0.0172 - val_acc: 0.9949
Q 322+7   T 329  ☑ 329 
Q 267-79  T 188  ☑ 188 
Q 767+461 T 1228 ☑ 1228
Q 363-220 T 143  ☑ 143 
Q 384+0   T 384  ☑ 384 
Q 561+109 T 670  ☑ 670 
Q 148+38  T 186  ☑ 186 
Q 778+92  T 870  ☑ 870 
Q 366-34  T 332  ☑ 332 
Q 389-56  T 333  ☑ 333 

--------------------------------------------------
Iteration 56
Train on 30375 samples, validate on 3375 samples
Epoch 1/1
30375/30375 [==============================] - 6s 210us/step - loss: 0.0011 - acc: 1.0000 - val_loss: 0.0179 - val_acc: 0.9949
Q 389-95  T 294  ☑ 294 
Q 82-45   T 37   ☑ 37  
Q 736-97  T 639  ☑ 639 
Q 833-437 T 396  ☑ 396 
Q 508+50  T 558  ☑ 558 
Q 284+251 T 535  ☑ 535 
Q 213+30  T 243  ☑ 243 
Q 474+28  T 502  ☑ 502 
Q 939-891 T 48   ☒ 57  
Q 485-7   T 478  ☑ 478 

--------------------------------------------------
Iteration 57
Train on 30375 samples, validate on 3375 samples
Epoch 1/1
30375/30375 [=====

30375/30375 [==============================] - 6s 209us/step - loss: 8.2254e-04 - acc: 1.0000 - val_loss: 0.0155 - val_acc: 0.9956
Q 376+11  T 387  ☑ 387 
Q 484+15  T 499  ☑ 499 
Q 97-81   T 16   ☑ 16  
Q 655-0   T 655  ☑ 655 
Q 377+6   T 383  ☑ 383 
Q 930+1   T 931  ☑ 931 
Q 440+63  T 503  ☑ 503 
Q 620+2   T 622  ☑ 622 
Q 474-88  T 386  ☑ 386 
Q 873-3   T 870  ☑ 870 

--------------------------------------------------
Iteration 70
Train on 30375 samples, validate on 3375 samples
Epoch 1/1
30375/30375 [==============================] - 6s 209us/step - loss: 6.8964e-04 - acc: 1.0000 - val_loss: 0.0154 - val_acc: 0.9959
Q 90+1    T 91   ☑ 91  
Q 238+1   T 239  ☑ 239 
Q 988+675 T 1663 ☑ 1663
Q 964-782 T 182  ☑ 182 
Q 806-753 T 53   ☑ 53  
Q 486-57  T 429  ☑ 429 
Q 954+229 T 1183 ☑ 1183
Q 208+6   T 214  ☑ 214 
Q 540-70  T 470  ☑ 470 
Q 213+30  T 243  ☑ 243 

--------------------------------------------------
Iteration 71
Train on 30375 samples, validate on 3375 samples
Epoch 1/1
30375/3037

30375/30375 [==============================] - 6s 209us/step - loss: 6.4458e-04 - acc: 1.0000 - val_loss: 0.0164 - val_acc: 0.9953
Q 56+23   T 79   ☑ 79  
Q 880+7   T 887  ☑ 887 
Q 290-18  T 272  ☑ 272 
Q 850+92  T 942  ☑ 942 
Q 229+7   T 236  ☑ 236 
Q 995-2   T 993  ☑ 993 
Q 600-85  T 515  ☑ 515 
Q 528+2   T 530  ☑ 530 
Q 794+510 T 1304 ☑ 1304
Q 614+6   T 620  ☑ 620 

--------------------------------------------------
Iteration 84
Train on 30375 samples, validate on 3375 samples
Epoch 1/1
30375/30375 [==============================] - 6s 209us/step - loss: 5.5362e-04 - acc: 1.0000 - val_loss: 0.0158 - val_acc: 0.9956
Q 173-37  T 136  ☑ 136 
Q 859-85  T 774  ☑ 774 
Q 951-855 T 96   ☑ 96  
Q 212-29  T 183  ☑ 183 
Q 25-22   T 3    ☑ 3   
Q 911-0   T 911  ☑ 911 
Q 804-9   T 795  ☑ 795 
Q 466-3   T 463  ☑ 463 
Q 991-7   T 984  ☑ 984 
Q 190-7   T 183  ☑ 183 

--------------------------------------------------
Iteration 85
Train on 30375 samples, validate on 3375 samples
Epoch 1/1
30375/3037

30375/30375 [==============================] - 6s 210us/step - loss: 4.0801e-04 - acc: 1.0000 - val_loss: 0.0173 - val_acc: 0.9955
Q 222-6   T 216  ☑ 216 
Q 76+20   T 96   ☑ 96  
Q 67-38   T 29   ☑ 29  
Q 876+3   T 879  ☑ 879 
Q 94-43   T 51   ☑ 51  
Q 816-338 T 478  ☑ 478 
Q 107+8   T 115  ☑ 115 
Q 156+22  T 178  ☑ 178 
Q 600-0   T 600  ☑ 600 
Q 246+8   T 254  ☑ 254 

--------------------------------------------------
Iteration 98
Train on 30375 samples, validate on 3375 samples
Epoch 1/1
30375/30375 [==============================] - 6s 210us/step - loss: 3.6085e-04 - acc: 1.0000 - val_loss: 0.0172 - val_acc: 0.9954
Q 181-82  T 99   ☑ 99  
Q 883+114 T 997  ☑ 997 
Q 628-5   T 623  ☑ 623 
Q 564-88  T 476  ☑ 476 
Q 301+180 T 481  ☑ 481 
Q 758+57  T 815  ☑ 815 
Q 131-81  T 50   ☑ 50  
Q 19+7    T 26   ☑ 26  
Q 669+24  T 693  ☑ 693 
Q 43+2    T 45   ☑ 45  

--------------------------------------------------
Iteration 99
Train on 30375 samples, validate on 3375 samples
Epoch 1/1
30375/3037

### Analysis
* addition & subtraction
* 3 digits
* Training size: 30.375K
* Total training epoch: 100 epoch
* First reach 99% validation accuracy at 25 epoch

In [16]:
DIGITS = 4
MAXLEN = DIGITS + 1 + DIGITS
ansmaxlen = DIGITS + 1
chars = '0123456789+- '
DATA_SIZE = 60000

ctable = CharacterTable(chars)
questions, expected, shows = DataGenerate('+-', maxlen=MAXLEN, digits=DIGITS, ansmaxlen=ansmaxlen)
x, y = Vectorization(questions, expected, ctable, maxlen=MAXLEN, ansmaxlen=ansmaxlen)
x_train, y_train, x_val, y_val, test_x, test_y = shuffle(x,y)

model = BuildModel(maxlen=MAXLEN, ansmaxlen=ansmaxlen)
for iteration in range(100):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train, verbose=1,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)

Generating data...
Total addition questions: 60000
Vectorization...
Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_11 (LSTM)               (None, 640)               1674240   
_________________________________________________________________
reshape_6 (Reshape)          (None, 5, 128)            0         
_________________________________________________________________
lstm_12 (LSTM)               (None, 5, 640)            1968640   
_________________________________________________________________
dense_6 (Dense)              (None, 5, 13)             8333      
Total params: 3,651,213
Trainable params: 3,651,213
Non-trainable params: 0
_________________________________________________________________

--------------------------------------------------
Iteration 0
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 15s 376us/s

40500/40500 [==============================] - 11s 278us/step - loss: 0.0749 - acc: 0.9758 - val_loss: 0.0794 - val_acc: 0.9721
Q 511-10    T 501   ☑ 501  
Q 6794+265  T 7059  ☒ 6059 
Q 62-1      T 61    ☑ 61   
Q 881+636   T 1517  ☑ 1517 
Q 4562-90   T 4472  ☑ 4472 
Q 8376+135  T 8511  ☑ 8511 
Q 9692-24   T 9668  ☑ 9668 
Q 850-57    T 793   ☑ 793  
Q 938+22    T 960   ☑ 960  
Q 3135+3    T 3138  ☑ 3138 

--------------------------------------------------
Iteration 13
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 11s 279us/step - loss: 0.0617 - acc: 0.9800 - val_loss: 0.0784 - val_acc: 0.9732
Q 3664+8    T 3672  ☑ 3672 
Q 7160+720  T 7880  ☑ 7880 
Q 724-4     T 720   ☑ 720  
Q 7119+8    T 7127  ☑ 7127 
Q 3279+6    T 3285  ☑ 3285 
Q 917-830   T 87    ☒ 77   
Q 564+461   T 1025  ☑ 1025 
Q 6766+0    T 6766  ☑ 6766 
Q 9555-663  T 8892  ☑ 8892 
Q 64+20     T 84    ☑ 84   

--------------------------------------------------
Iteratio

Q 3903-937  T 2966  ☑ 2966 
Q 760-712   T 48    ☒ 68   
Q 8033-83   T 7950  ☑ 7950 
Q 5378-3967 T 1411  ☒ 311  
Q 750-1     T 749   ☑ 749  
Q 8838-308  T 8530  ☑ 8530 
Q 892+437   T 1329  ☑ 1329 
Q 7873-49   T 7824  ☒ 7834 
Q 105+1     T 106   ☑ 106  
Q 3888+16   T 3904  ☑ 3904 

--------------------------------------------------
Iteration 26
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 11s 278us/step - loss: 0.0483 - acc: 0.9846 - val_loss: 0.0446 - val_acc: 0.9853
Q 8646+79   T 8725  ☑ 8725 
Q 9725+77   T 9802  ☑ 9802 
Q 114-48    T 66    ☑ 66   
Q 2024+475  T 2499  ☑ 2499 
Q 2482+13   T 2495  ☑ 2495 
Q 508-435   T 73    ☑ 73   
Q 5273+1    T 5274  ☑ 5274 
Q 2145+8    T 2153  ☑ 2153 
Q 456-21    T 435   ☑ 435  
Q 3862+806  T 4668  ☑ 4668 

--------------------------------------------------
Iteration 27
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 11s 279us/step - 

40500/40500 [==============================] - 11s 279us/step - loss: 0.0280 - acc: 0.9913 - val_loss: 0.0721 - val_acc: 0.9759
Q 132-83    T 49    ☑ 49   
Q 859+62    T 921   ☑ 921  
Q 7007+8    T 7015  ☑ 7015 
Q 8418+5    T 8423  ☑ 8423 
Q 417-1     T 416   ☑ 416  
Q 1404-0    T 1404  ☑ 1404 
Q 1513+659  T 2172  ☑ 2172 
Q 765+392   T 1157  ☒ 1057 
Q 912-77    T 835   ☑ 835  
Q 3997-341  T 3656  ☑ 3656 

--------------------------------------------------
Iteration 40
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 11s 277us/step - loss: 0.0231 - acc: 0.9929 - val_loss: 0.0394 - val_acc: 0.9876
Q 4473-5    T 4468  ☑ 4468 
Q 7788+2491 T 10279 ☒ 10289
Q 5911+436  T 6347  ☑ 6347 
Q 3412-0    T 3412  ☑ 3412 
Q 481-30    T 451   ☑ 451  
Q 279-56    T 223   ☑ 223  
Q 711+0     T 711   ☑ 711  
Q 543-57    T 486   ☑ 486  
Q 9505-42   T 9463  ☑ 9463 
Q 8946+20   T 8966  ☑ 8966 

--------------------------------------------------
Iteratio

40500/40500 [==============================] - 11s 280us/step - loss: 0.0063 - acc: 0.9985 - val_loss: 0.0348 - val_acc: 0.9895
Q 95-43     T 52    ☑ 52   
Q 982-0     T 982   ☑ 982  
Q 5606-40   T 5566  ☑ 5566 
Q 174-39    T 135   ☑ 135  
Q 44+11     T 55    ☑ 55   
Q 911-777   T 134   ☑ 134  
Q 58-7      T 51    ☑ 51   
Q 212-43    T 169   ☑ 169  
Q 3898+909  T 4807  ☒ 4817 
Q 331-4     T 327   ☑ 327  

--------------------------------------------------
Iteration 53
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 11s 278us/step - loss: 0.0016 - acc: 0.9999 - val_loss: 0.0296 - val_acc: 0.9914
Q 812+74    T 886   ☑ 886  
Q 1789+5    T 1794  ☑ 1794 
Q 6248-373  T 5875  ☑ 5875 
Q 647-6     T 641   ☑ 641  
Q 50-7      T 43    ☑ 43   
Q 7304-719  T 6585  ☑ 6585 
Q 928-186   T 742   ☑ 742  
Q 5534-5170 T 364   ☒ 336  
Q 93+51     T 144   ☑ 144  
Q 869-47    T 822   ☑ 822  

--------------------------------------------------
Iteratio

Q 1755+7    T 1762  ☑ 1762 
Q 74-8      T 66    ☑ 66   
Q 28-26     T 2     ☒ 1    
Q 5001-0    T 5001  ☑ 5001 
Q 967-60    T 907   ☑ 907  
Q 445+434   T 879   ☑ 879  
Q 723+42    T 765   ☑ 765  
Q 184-4     T 180   ☑ 180  
Q 528-39    T 489   ☑ 489  
Q 1718+76   T 1794  ☑ 1794 

--------------------------------------------------
Iteration 66
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 11s 278us/step - loss: 0.0051 - acc: 0.9987 - val_loss: 0.0326 - val_acc: 0.9904
Q 967-60    T 907   ☑ 907  
Q 840+110   T 950   ☑ 950  
Q 7160+41   T 7201  ☑ 7201 
Q 72-41     T 31    ☑ 31   
Q 162-21    T 141   ☑ 141  
Q 1133+4    T 1137  ☑ 1137 
Q 313+7     T 320   ☑ 320  
Q 146-62    T 84    ☑ 84   
Q 3589-3341 T 248   ☒ 237  
Q 931-29    T 902   ☑ 902  

--------------------------------------------------
Iteration 67
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 11s 279us/step - 

40500/40500 [==============================] - 11s 279us/step - loss: 0.0566 - acc: 0.9833 - val_loss: 0.0638 - val_acc: 0.9817
Q 202+63    T 265   ☑ 265  
Q 8159-225  T 7934  ☑ 7934 
Q 639-598   T 41    ☑ 41   
Q 40-27     T 13    ☑ 13   
Q 4552+331  T 4883  ☑ 4883 
Q 459-83    T 376   ☑ 376  
Q 8230-36   T 8194  ☑ 8194 
Q 4018-17   T 4001  ☑ 4001 
Q 80+37     T 117   ☑ 117  
Q 748-7     T 741   ☑ 741  

--------------------------------------------------
Iteration 80
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 11s 278us/step - loss: 0.0087 - acc: 0.9976 - val_loss: 0.0458 - val_acc: 0.9872
Q 535+2     T 537   ☑ 537  
Q 5736+95   T 5831  ☑ 5831 
Q 107+3     T 110   ☑ 110  
Q 629+72    T 701   ☑ 701  
Q 244-16    T 228   ☑ 228  
Q 264+69    T 333   ☑ 333  
Q 424+16    T 440   ☑ 440  
Q 5082+67   T 5149  ☑ 5149 
Q 40-33     T 7     ☑ 7    
Q 746-7     T 739   ☑ 739  

--------------------------------------------------
Iteratio

Q 9656-45   T 9611  ☑ 9611 
Q 5229-247  T 4982  ☑ 4982 
Q 82-3      T 79    ☑ 79   
Q 203-1     T 202   ☑ 202  
Q 628+75    T 703   ☑ 703  
Q 893+13    T 906   ☑ 906  
Q 8817+7082 T 15899 ☒ 15909
Q 149-42    T 107   ☑ 107  
Q 7165-17   T 7148  ☑ 7148 
Q 4689-726  T 3963  ☑ 3963 

--------------------------------------------------
Iteration 93
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 11s 279us/step - loss: 0.0053 - acc: 0.9984 - val_loss: 0.0579 - val_acc: 0.9844
Q 565+414   T 979   ☑ 979  
Q 259-257   T 2     ☒ 1    
Q 577-9     T 568   ☑ 568  
Q 7968-7    T 7961  ☑ 7961 
Q 9851+3    T 9854  ☑ 9854 
Q 695-1     T 694   ☑ 694  
Q 665+66    T 731   ☑ 731  
Q 5953+3116 T 9069  ☒ 8069 
Q 6333-742  T 5591  ☑ 5591 
Q 8607-834  T 7773  ☑ 7773 

--------------------------------------------------
Iteration 94
Train on 40500 samples, validate on 4500 samples
Epoch 1/1
40500/40500 [==============================] - 11s 278us/step - 

### Analysis
* addition & subtraction
* 4 digits
* Training size: 40.5K
* Total training epoch: 100 epoch
* First reach 99% validation accuracy at 49 epoch

# Multiplication

In [17]:
DIGITS = 3
MAXLEN = DIGITS + 1 + DIGITS
ansmaxlen = DIGITS + DIGITS
LAYERS = 3
chars = '0123456789* '
DATA_SIZE = 80000

ctable = CharacterTable(chars)
questions, expected, shows = DataGenerate('*', maxlen=MAXLEN, digits=DIGITS, ansmaxlen=ansmaxlen)
x, y = Vectorization(questions, expected, ctable, maxlen=MAXLEN, ansmaxlen=ansmaxlen)
x_train, y_train, x_val, y_val, test_x, test_y = shuffle(x,y)

model = BuildModel(maxlen=MAXLEN, ansmaxlen=ansmaxlen)
for iteration in range(100):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train, verbose=1,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)

Generating data...
Total addition questions: 80000
Vectorization...
Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_13 (LSTM)               (None, 768)               2399232   
_________________________________________________________________
reshape_7 (Reshape)          (None, 6, 128)            0         
_________________________________________________________________
lstm_14 (LSTM)               (None, 6, 768)            2755584   
_________________________________________________________________
dense_7 (Dense)              (None, 6, 12)             9228      
Total params: 5,164,044
Trainable params: 5,164,044
Non-trainable params: 0
_________________________________________________________________

--------------------------------------------------
Iteration 0
Train on 54000 samples, validate on 6000 samples
Epoch 1/1
54000/54000 [==============================] - 24s 444us/s

54000/54000 [==============================] - 19s 358us/step - loss: 0.6226 - acc: 0.7580 - val_loss: 0.6580 - val_acc: 0.7386
Q 698*673 T 469754 ☒ 442754
Q 81*72   T 5832   ☒ 5822  
Q 649*637 T 413413 ☒ 413693
Q 180*40  T 7200   ☑ 7200  
Q 613*247 T 151411 ☒ 159491
Q 187*1   T 187    ☑ 187   
Q 810*709 T 574290 ☒ 586490
Q 532*97  T 51604  ☒ 51124 
Q 132*26  T 3432   ☒ 3452  
Q 906*589 T 533634 ☒ 533494

--------------------------------------------------
Iteration 13
Train on 54000 samples, validate on 6000 samples
Epoch 1/1
54000/54000 [==============================] - 19s 359us/step - loss: 0.5982 - acc: 0.7688 - val_loss: 0.6433 - val_acc: 0.7463
Q 162*91  T 14742  ☒ 15742 
Q 992*199 T 197408 ☒ 198808
Q 456*268 T 122208 ☒ 124268
Q 992*767 T 760864 ☒ 769204
Q 861*33  T 28413  ☒ 28973 
Q 997*145 T 144565 ☒ 145965
Q 94*48   T 4512   ☑ 4512  
Q 359*83  T 29797  ☒ 30697 
Q 212*18  T 3816   ☒ 3356  
Q 248*86  T 21328  ☒ 21968 

--------------------------------------------------
Iteratio

Q 734*336 T 246624 ☒ 240084
Q 682*53  T 36146  ☒ 35626 
Q 301*28  T 8428   ☑ 8428  
Q 168*26  T 4368   ☒ 4288  
Q 125*82  T 10250  ☒ 10300 
Q 846*8   T 6768   ☑ 6768  
Q 894*465 T 415710 ☒ 417310
Q 764*187 T 142868 ☒ 145598
Q 545*21  T 11445  ☒ 11485 
Q 53*6    T 318    ☑ 318   

--------------------------------------------------
Iteration 26
Train on 54000 samples, validate on 6000 samples
Epoch 1/1
54000/54000 [==============================] - 19s 360us/step - loss: 0.2345 - acc: 0.9252 - val_loss: 0.7878 - val_acc: 0.7619
Q 952*36  T 34272  ☒ 34832 
Q 985*80  T 78800  ☑ 78800 
Q 873*783 T 683559 ☒ 683899
Q 162*73  T 11826  ☒ 11706 
Q 769*7   T 5383   ☑ 5383  
Q 829*130 T 107770 ☒ 106370
Q 209*88  T 18392  ☑ 18392 
Q 476*66  T 31416  ☒ 30016 
Q 501*6   T 3006   ☑ 3006  
Q 950*31  T 29450  ☒ 29850 

--------------------------------------------------
Iteration 27
Train on 54000 samples, validate on 6000 samples
Epoch 1/1
54000/54000 [==============================] - 19s 358us/step - 

54000/54000 [==============================] - 19s 358us/step - loss: 0.0404 - acc: 0.9922 - val_loss: 1.1066 - val_acc: 0.7738
Q 592*90  T 53280  ☑ 53280 
Q 165*0   T 0      ☑ 0     
Q 55*39   T 2145   ☒ 2165  
Q 934*402 T 375468 ☒ 376128
Q 92*29   T 2668   ☒ 2688  
Q 921*26  T 23946  ☑ 23946 
Q 791*79  T 62489  ☒ 62589 
Q 575*399 T 229425 ☒ 231125
Q 508*311 T 157988 ☒ 157798
Q 63*12   T 756    ☑ 756   

--------------------------------------------------
Iteration 40
Train on 54000 samples, validate on 6000 samples
Epoch 1/1
54000/54000 [==============================] - 19s 359us/step - loss: 0.0487 - acc: 0.9892 - val_loss: 1.1408 - val_acc: 0.7682
Q 471*40  T 18840  ☑ 18840 
Q 972*54  T 52488  ☒ 52948 
Q 428*39  T 16692  ☒ 16392 
Q 461*57  T 26277  ☑ 26277 
Q 163*90  T 14670  ☑ 14670 
Q 261*68  T 17748  ☒ 17948 
Q 894*303 T 270882 ☒ 275382
Q 238*75  T 17850  ☒ 17650 
Q 733*325 T 238225 ☒ 231725
Q 846*357 T 302022 ☒ 310122

--------------------------------------------------
Iteratio

54000/54000 [==============================] - 20s 362us/step - loss: 0.0307 - acc: 0.9927 - val_loss: 1.2542 - val_acc: 0.7757
Q 609*487 T 296583 ☒ 299183
Q 873*758 T 661734 ☒ 659914
Q 974*776 T 755824 ☒ 757064
Q 225*141 T 31725  ☒ 31325 
Q 215*152 T 32680  ☒ 32880 
Q 200*39  T 7800   ☑ 7800  
Q 169*118 T 19942  ☒ 29862 
Q 693*390 T 270270 ☒ 270870
Q 146*68  T 9928   ☒ 10288 
Q 312*249 T 77688  ☒ 79288 

--------------------------------------------------
Iteration 53
Train on 54000 samples, validate on 6000 samples
Epoch 1/1
54000/54000 [==============================] - 19s 358us/step - loss: 0.0107 - acc: 0.9987 - val_loss: 1.2626 - val_acc: 0.7788
Q 218*47  T 10246  ☒ 1008  
Q 430*64  T 27520  ☒ 26920 
Q 790*71  T 56090  ☑ 56090 
Q 603*49  T 29547  ☒ 29347 
Q 605*214 T 129470 ☒ 129420
Q 419*11  T 4609   ☒ 4589  
Q 976*958 T 935008 ☒ 945508
Q 973*519 T 504987 ☒ 507127
Q 538*98  T 52724  ☑ 52724 
Q 896*657 T 588672 ☒ 580272

--------------------------------------------------
Iteratio

Q 819*89  T 72891  ☒ 73791 
Q 522*34  T 17748  ☒ 17348 
Q 695*93  T 64635  ☒ 64935 
Q 904*224 T 202496 ☒ 203396
Q 567*74  T 41958  ☒ 41358 
Q 591*149 T 88059  ☒ 86759 
Q 486*4   T 1944   ☑ 1944  
Q 352*6   T 2112   ☑ 2112  
Q 873*716 T 625068 ☒ 627468
Q 935*250 T 233750 ☑ 233750

--------------------------------------------------
Iteration 66
Train on 54000 samples, validate on 6000 samples
Epoch 1/1
54000/54000 [==============================] - 19s 359us/step - loss: 0.0244 - acc: 0.9936 - val_loss: 1.3867 - val_acc: 0.7764
Q 756*66  T 49896  ☒ 49696 
Q 771*480 T 370080 ☒ 372080
Q 945*59  T 55755  ☒ 55955 
Q 107*70  T 7490   ☒ 7590  
Q 591*529 T 312639 ☒ 303239
Q 656*93  T 61008  ☒ 60728 
Q 652*388 T 252976 ☒ 255276
Q 396*74  T 29304  ☒ 29784 
Q 890*199 T 177110 ☒ 177510
Q 780*0   T 0      ☑ 0     

--------------------------------------------------
Iteration 67
Train on 54000 samples, validate on 6000 samples
Epoch 1/1
54000/54000 [==============================] - 19s 359us/step - 

54000/54000 [==============================] - 19s 360us/step - loss: 5.4093e-04 - acc: 1.0000 - val_loss: 1.4544 - val_acc: 0.7823
Q 975*60  T 58500  ☑ 58500 
Q 689*200 T 137800 ☑ 137800
Q 413*21  T 8673   ☑ 8673  
Q 419*87  T 36453  ☒ 36413 
Q 812*598 T 485576 ☒ 486956
Q 951*539 T 512589 ☒ 515189
Q 386*3   T 1158   ☑ 1158  
Q 496*69  T 34224  ☑ 34224 
Q 197*26  T 5122   ☒ 5022  
Q 988*447 T 441636 ☒ 440316

--------------------------------------------------
Iteration 80
Train on 54000 samples, validate on 6000 samples
Epoch 1/1
54000/54000 [==============================] - 19s 358us/step - loss: 4.6936e-04 - acc: 1.0000 - val_loss: 1.4649 - val_acc: 0.7823
Q 947*6   T 5682   ☑ 5682  
Q 328*134 T 43952  ☒ 44552 
Q 123*57  T 7011   ☒ 6991  
Q 347*15  T 5205   ☑ 5205  
Q 80*58   T 4640   ☑ 4640  
Q 320*44  T 14080  ☒ 14880 
Q 621*46  T 28566  ☑ 28566 
Q 90*55   T 4950   ☑ 4950  
Q 747*19  T 14193  ☒ 14413 
Q 495*4   T 1980   ☑ 1980  

--------------------------------------------------


54000/54000 [==============================] - 19s 358us/step - loss: 9.8626e-04 - acc: 1.0000 - val_loss: 1.4298 - val_acc: 0.7815
Q 199*136 T 27064  ☒ 25664 
Q 727*40  T 29080  ☑ 29080 
Q 637*17  T 10829  ☒ 10869 
Q 241*6   T 1446   ☑ 1446  
Q 985*913 T 899305 ☒ 898245
Q 767*76  T 58292  ☒ 58552 
Q 640*591 T 378240 ☒ 381640
Q 705*48  T 33840  ☒ 33440 
Q 710*0   T 0      ☑ 0     
Q 543*36  T 19548  ☒ 19148 

--------------------------------------------------
Iteration 93
Train on 54000 samples, validate on 6000 samples
Epoch 1/1
54000/54000 [==============================] - 19s 358us/step - loss: 8.2759e-04 - acc: 1.0000 - val_loss: 1.4395 - val_acc: 0.7821
Q 855*140 T 119700 ☒ 117700
Q 150*12  T 1800   ☑ 1800  
Q 918*425 T 390150 ☒ 393750
Q 846*8   T 6768   ☑ 6768  
Q 678*67  T 45426  ☒ 45726 
Q 431*337 T 145247 ☒ 149207
Q 363*48  T 17424  ☒ 17304 
Q 272*5   T 1360   ☑ 1360  
Q 704*493 T 347072 ☒ 349252
Q 511*75  T 38325  ☒ 38225 

--------------------------------------------------


### Analysis
* Multiplication
* 3 digits
* Need to modify the maximum length of answer
* Training size: 54K
* Total training epoch: 100 epoch
* First reach 75% validation accuracy at 15 epoch
* Best accuracy: 78.30%

# Testing

In [18]:
print("MSG : Prediction")
for i in range(10):
    ind = np.random.randint(0, len(test_x))
    rowx, rowy = test_x[np.array([ind])], test_y[np.array([ind])]
    preds = model.predict_classes(rowx, verbose=0)
    q = ctable.decode(rowx[0])
    correct = ctable.decode(rowy[0])
    guess = ctable.decode(preds[0], calc_argmax=False)
    print('Q', q[::-1] if REVERSE else q, end=' ')
    print('T', correct, end=' ')
    if correct == guess:
        print(colors.ok + '☑' + colors.close, end=' ')
    else:
        print(colors.fail + '☒' + colors.close, end=' ')
    print(guess)

MSG : Prediction
Q 896*74  T 66304  ☑ 66304 
Q 319*60  T 19140  ☑ 19140 
Q 285*77  T 21945  ☒ 21245 
Q 610*80  T 48800  ☑ 48800 
Q 591*232 T 137112 ☒ 138212
Q 983*757 T 744131 ☒ 734571
Q 146*27  T 3942   ☒ 3982  
Q 560*219 T 122640 ☒ 126840
Q 938*7   T 6566   ☑ 6566  
Q 667*76  T 50692  ☒ 50582 


# Report 
* Addition only or substraction only or addition+substraction both can reach 99% evaluate accuracy
* The more digits need to be computed, the more training data need to make accuracy higher
* Multiplication is more difficult than addition+substraction. Even use 54K training data to train, just reach almost 80% validation accuracy